  <font color='green' size=5> HW #1:  Populate an analytic file using the raw claims data file </font> 

**Healthcare Case Studies ** <br>
**ANLT 272** <br>
**Apr 10, 2018** <br>
<br>

<font color='green' size=4> By: Alan Kuang, Ali Taheri</font>

In [218]:
import pandas as pd
import numpy as np

#import data sets into pandas dataframes and view the first 5 rows

#enroll file
enroll_df = pd.read_csv('uopjan16_enroll_in.csv')

#removing duplicate rows
enroll_df=enroll_df.drop_duplicates()

enroll_df.head(3)

,pat_id,estring,clm_frst,clm_last,enr_frst,enr_last,mxce_fst,mxce_lst,der_sex,der_yob,nbr_clms,mon_totl,pat_region
0,0309AAAAAAAATOHM,----------------------------------------------...,2001-02-19,2015-09-30,2001-01-01,2015-09-30,2001-01-01,2015-09-30,M,1949.0,1016,149.0,MW
1,0309AAAAAAAEEDAZ,----------------------------------------------...,2001-01-08,2015-09-30,2001-01-01,2015-09-30,2001-01-01,2015-09-30,F,1944.0,1091,177.0,MW
2,0309AAAAAAAFHCZL,----------------------------------------------...,2001-01-09,2015-09-30,2001-01-01,2015-09-30,2001-01-01,2015-09-30,M,1932.0,1130,177.0,MW


In [219]:
#claims file
claims_df = pd.read_csv('uopjan16_claims_in.csv',dtype={"ndc": object})

#changing type of dates to datetime
claims_df['from_dt'] =  pd.to_datetime(claims_df['from_dt'], format='%Y-%m-%d')
claims_df['to_dt'] =  pd.to_datetime(claims_df['to_dt'], format='%Y-%m-%d')

#removing leading 0's from ndc
claims_df['ndc'] = claims_df['ndc'].apply(lambda x: str(x).lstrip('0'))

#removing prescriptions claims without DAYSSUP
claims_df=claims_df[(claims_df['rectype'] != 'P') | (claims_df['DAYSSUP'] > 0)]

#removing duplicate rows
claims_df=claims_df.drop_duplicates()

claims_df.head(3)

,pat_id,rec_spec,rectype,ALLOWED,DAYSSUP,QUAN,rec_ix,from_dt,to_dt,prov_id,...,ndc,proc_cde,diag1,diag2,diag3,diag4,ptypeflg,pos,cluspvid,conf_num
0,0309AAAAAAAATOHM,NaN,P,138.84,90.0,180.0,1200779,2011-08-03,2011-08-03,0309AAAAAAAAAXSH,...,591397201,NaN,NaN,NaN,NaN,NaN,2,00,NaN,NaN
1,0309AAAAAAAATOHM,NaN,P,73.98,90.0,90.0,1200780,2011-08-03,2011-08-03,0309AAAAAAAAAXSH,...,62037083001,NaN,NaN,NaN,NaN,NaN,2,00,NaN,NaN
2,0309AAAAAAAATOHM,NaN,P,107.10,90.0,90.0,1200781,2011-08-17,2011-08-17,0309AAAAAAAAAXSH,...,68180048103,NaN,NaN,NaN,NaN,NaN,2,00,0309AAAAAAAAAGEP,NaN


In [220]:
#lookup file
lookup_df = pd.read_csv('uopjanref_rx_lookup.csv')

#removing leading 0's from National Drug Code
lookup_df['National Drug Code'] = lookup_df['National Drug Code'].apply(lambda x: str(x).lstrip('0'))

#removing duplicate rows
lookup_df = lookup_df.drop_duplicates()

lookup_df.head(3)

,National Drug Code,ahfscc,tee_code,route_admin,brand_indicator,rx_otc,last_chg_dt,brand_name,gpi,rx_group,drug_class,gen_nm,fd_drug_name,manufacturer_name,dosage_form,unit,strength,usc,usc_desc
0,2197590,680800,NOT RATED,TRANSDERMAL,TRADEMARK,RX,30-Dec-15,AXIRON,2.31E+13,ANDROGENS-ANABOLIC,ANDROGENS,TESTOSTERONE,TESTOSTERONE TD SOLN 30 MG/ACT,LILLY,SOLUTION,MG/ACT,30.0,52153,"HORMONES,ANDROGENS,TRNSDR"
1,2322730,289200,BIOEQUIVALENCE RESOLVED,ORAL,TRADEMARK,RX,30-Dec-15,STRATTERA,6.1354E+13,ADHD/ANTI-NARCOLEPSY/ANTI-OBESITY/ANOREXIANTS,ATTENTION-DEFICIT/HYPERACTIVITY DISORDER (ADHD...,ATOMOXETINE HCL,ATOMOXETINE HCL CAP 10 MG (BASE EQUIV),LILLY,CAPSULE,MG,10.0,64700,NEWER GEN PSYCOTHER AGT
2,2322830,289200,BIOEQUIVALENCE RESOLVED,ORAL,TRADEMARK,RX,30-Dec-15,STRATTERA,6.1354E+13,ADHD/ANTI-NARCOLEPSY/ANTI-OBESITY/ANOREXIANTS,ATTENTION-DEFICIT/HYPERACTIVITY DISORDER (ADHD...,ATOMOXETINE HCL,ATOMOXETINE HCL CAP 25 MG (BASE EQUIV),LILLY,CAPSULE,MG,25.0,64700,NEWER GEN PSYCOTHER AGT


In [221]:
#Betablocker sheet
betablocker_df=pd.read_csv('HMG_CO-A.csv')

#removing leading 0's from National Drug Code
betablocker_df['National Drug Code'] = betablocker_df['National Drug Code'].apply(lambda x: str(x).lstrip('0'))

#removing duplicate rows
betablocker_df = betablocker_df.drop_duplicates()

betablocker_df.head(3)

,National Drug Code,brand_name,gen_nm
0,2477090,LIVALO,PITAVASTATIN CALCIUM
1,2477190,LIVALO,PITAVASTATIN CALCIUM
2,2477290,LIVALO,PITAVASTATIN CALCIUM


In [222]:
#Adding a column to claims_df that shows prescribed drug is beta-blocker
claims_df['beta_blocker']= np.where(claims_df['ndc'].isin(betablocker_df['National Drug Code']),'1','0')

In [223]:
#A dataframe for making analytic file
df = pd.DataFrame(data= enroll_df['pat_id'])
df=df.rename(index=str,columns={"pat_id": "PatID"})
df.head(3)

,PatID
0,0309AAAAAAAATOHM
1,0309AAAAAAAEEDAZ
2,0309AAAAAAAFHCZL


In [224]:
#making a series of ndc's of all AD and AC drugs
AD_ndc_series=lookup_df.loc[lookup_df['rx_group'] == 'ANTIDIABETICS' ,'National Drug Code']
AC_ndc_series= lookup_df.loc[lookup_df['rx_group'] == 'ANTICOAGULANTS','National Drug Code']

In [225]:
claims_df['Drug_Type']=np.where( (claims_df['rectype'] == 'P') &\
                                (claims_df['ndc'].isin(AD_ndc_series)) , 'AD' , 'Neither')

claims_df.loc[ (claims_df['rectype'] == 'P') & (claims_df['ndc'].isin(AC_ndc_series)) , 'Drug_Type'] ='AC' 

In [226]:
#a function for checking diagnosis of dyslipidemia
def lipidmatch(series):
    return series.apply(lambda str:  str in {'272','2720','2721','2722', '2723', '2724','2725','2726', '2727','2728','2729'})
 

In [227]:
from datetime import datetime
tstart = datetime.now()
for row in df.index:
    
    #Extracting a dataframe for each patient with desired rectypes
    patient_df=claims_df[(claims_df['pat_id'] == df.loc[row,'PatID']) & \
                         (claims_df['rectype'].isin({'P','M','S','F'}))]
    
    #Extracting a dataframe with all prescription claims between Jan 1 and March 31
    respective_df= patient_df[ (patient_df['rectype'] == 'P')                  & \
                               (patient_df['from_dt'] <= '2013-3-31')          & \
                               (patient_df['from_dt'] >= '2013-1-1')]
     
    
    #Escaping patients without prescription claim
    if (len(respective_df) == 0):
        continue
        
    #index date and index group
    drug_types_for_parient=respective_df['Drug_Type'].unique()
    if (len(drug_types_for_parient) == 1):
        df.loc[row,'IDX_Date']= respective_df['from_dt'].min()
        df.loc[row,'IDX_Group']= drug_types_for_parient[0]
    else:
        df.loc[row,'IDX_Date']= respective_df.loc[ respective_df['Drug_Type'] != 'Neither','from_dt'].min()
        drug_types_at_index_date=respective_df.loc[respective_df['from_dt'] == df.loc[row,'IDX_Date'] ,'Drug_Type'].unique()  
        if   (('AC' in  drug_types_at_index_date) & ('AD' not in  drug_types_at_index_date)):  
            df.loc[row,'IDX_Group']= 'AC'
        elif (('AC' not in  drug_types_at_index_date) & ('AD' in  drug_types_at_index_date)):
            df.loc[row,'IDX_Group']= 'AD'
        else:
            df.loc[row,'IDX_Group']= 'DC'
                
    #Index drug generic name
    index_drug_code = respective_df.loc[respective_df['from_dt'] == df.loc[row,'IDX_Date'],'ndc'].iloc[0]
    if (df.loc[row,'IDX_Group'] == 'DC'):
        index_drug_code = respective_df.loc[(respective_df['from_dt'] == df.loc[row,'IDX_Date']) &\
                                      (respective_df['Drug_Type'] == 'AD') ,'ndc'].iloc[0]      
    df.loc[row,'IDX_gen_name'] = lookup_df.loc[lookup_df['National Drug Code'] == index_drug_code,'gen_nm'].iloc[0]
    
    #Year of birth
    df.loc[row , 'YOB'] = enroll_df.loc[enroll_df['pat_id'] == df.loc[row, 'PatID'] ,'der_yob'].iloc[0]
    
    #Age at index
    df.loc[row , 'IDX_Age'] = 2013 - df.loc[row , 'YOB']
    
    #Gender   
    sex= enroll_df.loc[enroll_df['pat_id'] == df.loc[row, 'PatID'],'der_sex']
    df.loc[str(row) , 'Pat_sex'] = np.where( sex == 'M', '0' , '1').item()
    
    #Pre index hospitilization
    pre_hosp_status=((patient_df['rectype'] == 'F')                 & \
                     (patient_df['from_dt'] < patient_df['to_dt'])   & \
                     (patient_df['to_dt'] < df.loc[row , 'IDX_Date'])).unique()
    df.loc[row , 'Pre_hosp'] = np.where( True in pre_hosp_status, '1' , '0').item()
    
    
    #Pre index Cholesterol abnormality
    dislipidemia_status = ( (patient_df['rectype'] != 'P') & \
                           (patient_df['from_dt'] < df.loc[row ,'IDX_Date'])  & \
                           ( lipidmatch(patient_df['diag1']) | lipidmatch(patient_df['diag2']) | \
                             lipidmatch(patient_df['diag3']) | lipidmatch(patient_df['diag3'])) ).unique()
    df.loc[row , 'Pre_Dyslipidemia'] = np.where( True in dislipidemia_status, '1' , '0').item()
    
    #Pre index beta blocker Rx flag
    pre_betablocker_status = ((patient_df['rectype'] == 'P')     & \
                              (patient_df['beta_blocker'] == '1') & \
                              (patient_df['from_dt'] < df.loc[row ,'IDX_Date']) ).unique()
    df.loc[row , 'Pre_HMG_CO-A'] = np.where( True in pre_betablocker_status, '1' , '0').item()
    
    #Number of post index prescriptions for index drug
    post_index_df= patient_df[(patient_df['rectype'] == 'P') &\
                                  (patient_df['ndc'] == index_drug_code)  & \
                                  (patient_df['from_dt'] >= df.loc[row , 'IDX_Date']) &\
                                  (patient_df['from_dt'] <= df.loc[row , 'IDX_Date'] + pd.Timedelta('180 days'))]
    
    df.loc[row , 'Post_IDX_Rx'] = len(post_index_df)
    
    #Days' supply of index drug
    df.loc[row , 'Post_IDX_DS'] = post_index_df['DAYSSUP'].sum()
    
    #Proportion of days covered for index drug
    df.loc[row , 'IDX_PDC'] = df.loc[row , 'Post_IDX_DS'] / 180
        
    #The Proportion of patients that had a discontinuation of the index drug
    post_index_df = post_index_df.sort_values(by=['from_dt'])
    post_index_df = post_index_df.reset_index()
    IDX_Discon = 0
    if (post_index_df['DAYSSUP'].sum() < 180):
        IDX_Discon = 1
    elif (len(post_index_df) > 1):
        for inx in range(len(post_index_df)-1):
            if (post_index_df.loc[inx+1,'from_dt'] - post_index_df.loc[inx,'from_dt'] -\
                pd.Timedelta(days=post_index_df.loc[inx,'DAYSSUP']) >= pd.Timedelta(days= 30)):
                IDX_Discon = 1
    df.loc[row , 'IDX_Discon'] = IDX_Discon
    
    #Flag for PDC>= 0.80
    df.loc[row , 'IDX_PDC>80'] = np.where( df.loc[row , 'IDX_PDC'] >= .8, '1' , '0').item()
    
    #No Post index hospitilization
    post_hosp_status=((patient_df['rectype'] == 'F') &\
                      (patient_df['to_dt'] > df.loc[row, 'IDX_Date']) &\
                      (patient_df['to_dt'] > patient_df['from_dt']) &\
                      (patient_df['to_dt'] <= (df.loc[row, 'IDX_Date'] +\
                                                          pd.Timedelta('180 days')))).unique()
    df.loc[str(row) , 'Post_hosp_neg'] = np.where( True in post_hosp_status, '0' , '1').item()
    
        
tend= datetime.now()

tend - tstart 

datetime.timedelta(0, 879, 736790)

In [228]:
#finding number of patients per each index group
quantities=df.groupby('IDX_Group')['IDX_Group'].count()
quantities

IDX_Group
AC          806
AD         4958
DC           28
Neither       5
Name: IDX_Group, dtype: int64

In [229]:
#finding number of male patients per each index group
Males_qty = df[df['Pat_sex'] == '0'].groupby('IDX_Group')['IDX_Group'].count()
Males_qty


IDX_Group
AC          434
AD         2401
DC           14
Neither       1
Name: IDX_Group, dtype: int64

In [230]:
#finding number of patients younger than 18 years old per each index group
younger_18 = df[(0 <= df['IDX_Age']) & ( df['IDX_Age'] <= 18)].groupby('IDX_Group')['IDX_Group'].count()
younger_18

IDX_Group
AC      5
AD    136
Name: IDX_Group, dtype: int64

In [231]:
#finding number of patients with age between 19 and 50 years old per each index group
between_19_50 = df[(19 <= df['IDX_Age']) & ( df['IDX_Age'] <= 50)].groupby('IDX_Group')['IDX_Group'].count() 
between_19_50

IDX_Group
AC          122
AD         1357
DC            7
Neither       4
Name: IDX_Group, dtype: int64

In [232]:
#finding number of patients with age between 51 and 65 years old per each index group
between_51_65 = df[(51 <= df['IDX_Age']) & ( df['IDX_Age'] <= 65)].groupby('IDX_Group')['IDX_Group'].count()
between_51_65

IDX_Group
AC     310
AD    2419
DC      13
Name: IDX_Group, dtype: int64

In [233]:
#finding number of patients with age between 66 and 75 years old per each index group
between_66_75 = df[(66 <= df['IDX_Age']) & ( df['IDX_Age'] <= 75)].groupby('IDX_Group')['IDX_Group'].count()
between_66_75

IDX_Group
AC    148
AD    682
DC      5
Name: IDX_Group, dtype: int64

In [234]:
#finding number of patients older than 76 years old per each index group
older_76 = df[76 <= df['IDX_Age']].groupby('IDX_Group')['IDX_Group'].count() 
older_76

IDX_Group
AC         221
AD         362
DC           3
Neither      1
Name: IDX_Group, dtype: int64

In [235]:
#number of paients with history of hospitalization before index date per each index group
Pre_hosp = df[df['Pre_hosp'] == '1'].groupby('IDX_Group')['IDX_Group'].count() 
Pre_hosp

IDX_Group
AC         269
AD         632
DC          10
Neither      2
Name: IDX_Group, dtype: int64

In [236]:
#number of paients with diagnosis of dyslipidemia before index date per each index group
Pre_Dyslipidemia = df[df['Pre_Dyslipidemia'] == '1'].groupby('IDX_Group')['IDX_Group'].count()
Pre_Dyslipidemia

IDX_Group
AC          358
AD         2689
DC           17
Neither       2
Name: IDX_Group, dtype: int64

In [237]:
#number of paients with prescription claim of beta blocker before index date per each index group
Pre_HMG_CO_A = df[df['Pre_HMG_CO-A'] == '1'].groupby('IDX_Group')['IDX_Group'].count()
Pre_HMG_CO_A

IDX_Group
AC          375
AD         2784
DC           15
Neither       1
Name: IDX_Group, dtype: int64

In [238]:
#mean and standard deviation of Post_IDX_Rx
Post_IDX_Rx_mean = df.groupby(['IDX_Group'])['Post_IDX_Rx'].mean()
print(Post_IDX_Rx_mean)

Post_IDX_Rx_std = df.groupby(['IDX_Group'])['Post_IDX_Rx'].std()
print(Post_IDX_Rx_std)

IDX_Group
AC         3.227047
AD         3.207543
DC         3.607143
Neither    1.800000
Name: Post_IDX_Rx, dtype: float64
IDX_Group
AC         2.089147
AD         2.048703
DC         2.131584
Neither    0.836660
Name: Post_IDX_Rx, dtype: float64


In [239]:
#mean and standard deviation of Post_IDX_DS
Post_IDX_DS_mean =df.groupby(['IDX_Group'])['Post_IDX_DS'].mean()
print(Post_IDX_DS_mean)

Post_IDX_DS_std=df.groupby(['IDX_Group'])['Post_IDX_DS'].std()
print(Post_IDX_DS_std)

IDX_Group
AC         131.383375
AD         131.549818
DC         135.321429
Neither     52.000000
Name: Post_IDX_DS, dtype: float64
IDX_Group
AC         79.547248
AD         75.893382
DC         68.478736
Neither    27.748874
Name: Post_IDX_DS, dtype: float64


In [240]:
#mean and standard deviation of IDX_PDC
IDX_PDC_mean = df.groupby(['IDX_Group'])['IDX_PDC'].mean()
print(IDX_PDC_mean)

IDX_PDC_std= df.groupby(['IDX_Group'])['IDX_PDC'].std()
print(IDX_PDC_std)

IDX_Group
AC         0.729908
AD         0.730832
DC         0.751786
Neither    0.288889
Name: IDX_PDC, dtype: float64
IDX_Group
AC         0.441929
AD         0.421630
DC         0.380437
Neither    0.154160
Name: IDX_PDC, dtype: float64


In [241]:
#mean and standard deviation of IDX_Discon
IDX_Discon_mean = df.groupby(['IDX_Group'])['IDX_Discon'].mean()
print(IDX_Discon_mean)

IDX_Discon_std = df.groupby(['IDX_Group'])['IDX_Discon'].std()
print(IDX_Discon_std)

IDX_Group
AC         0.619107
AD         0.640783
DC         0.571429
Neither    1.000000
Name: IDX_Discon, dtype: float64
IDX_Group
AC         0.485908
AD         0.479819
DC         0.503953
Neither    0.000000
Name: IDX_Discon, dtype: float64


In [242]:
#number of paients with IDX_PDC greater than .8 per each index group
IDX_PDC_80 = df[df['IDX_PDC>80'] == '1'].groupby('IDX_Group')['IDX_Group'].count()
IDX_PDC_80


IDX_Group
AC     400
AD    2376
DC      15
Name: IDX_Group, dtype: int64

In [243]:
#number of paients with no hospitalization after index date per each index group
Post_hosp_neg = df[df['Post_hosp_neg'] == '1'].groupby('IDX_Group')['IDX_Group'].count()
Post_hosp_neg

IDX_Group
AC          699
AD         4680
DC           27
Neither       4
Name: IDX_Group, dtype: int64

In [244]:
#a function for returning value of 0 for indexes not in a pandas series
def value(series, inx):
    if (inx in series.index):
        return series[inx]
    else:
        return 0

In [245]:
#A dataframe for storing numbers and percents of different festures (At, Pre and Post)
index_group_n = {'AD':None , 'AC':None , 'DC':None ,'Neither':None}
index_group_p = {'AD %':None , 'AC %':None , 'DC %':None ,'Neither %':None}


for index_group in index_group_n.keys():
    index_group_n[index_group]=pd.Series([quantities[index_group] , \
                              value(Males_qty,index_group) ,\
                              value(younger_18,index_group) ,\
                              value(between_19_50,index_group) ,\
                              value(between_51_65,index_group) ,\
                              value(between_66_75,index_group) ,\
                              value(older_76,index_group) ,\
                              value(Pre_hosp ,index_group) ,\
                              value(Pre_Dyslipidemia,index_group) ,\
                              value(Pre_HMG_CO_A,index_group) ,\
                              value(IDX_PDC_80,index_group) ,\
                              value(Post_hosp_neg,index_group) ,\
                              value(Post_IDX_Rx_mean,index_group),\
                              value(Post_IDX_DS_mean,index_group),\
                              value(IDX_PDC_mean,index_group) ,\
                              value(IDX_Discon_mean,index_group) ])

    index_group_p[index_group+' %'] = 100*index_group_n[index_group] \
                                            /quantities[index_group] 

result = pd.concat([index_group_n['AD'], index_group_p['AD %'], \
                    index_group_n['AC'] ,index_group_p['AC %'], \
                    index_group_n['DC'] , index_group_p['DC %'], \
                    index_group_n['Neither'], index_group_p['Neither %'] ], axis=1)

post_index_stat=round(result.iloc[12:16,],2)
result=result.iloc[0:12,]

for column_num in range(4):
    result.iloc[:,2*column_num]=result.iloc[:,2*column_num].astype('int')
    result.iloc[:,2*column_num + 1]=round(result.iloc[:,2*column_num +1] ,2)
    
result

,0,1,2,3,4,5,6,7
0,4958,100.00,806,100.00,28,100.00,5,100.0
1,2401,48.43,434,53.85,14,50.00,1,20.0
2,136,2.74,5,0.62,0,0.00,0,0.0
3,1357,27.37,122,15.14,7,25.00,4,80.0
4,2419,48.79,310,38.46,13,46.43,0,0.0
5,682,13.76,148,18.36,5,17.86,0,0.0
6,362,7.30,221,27.42,3,10.71,1,20.0
7,632,12.75,269,33.37,10,35.71,2,40.0
8,2689,54.24,358,44.42,17,60.71,2,40.0
9,2784,56.15,375,46.53,15,53.57,1,20.0


In [246]:
#A dataframe for storing mean and standard deviation of different festures (Post)
post_index_mean = {'AD':None , 'AC':None , 'DC':None ,'Neither':None}
post_index_std = {'AD':None , 'AC':None , 'DC':None ,'Neither':None}


for index_group in post_index_mean.keys():
    post_index_mean[index_group]=pd.Series([ \
                              value(Post_IDX_Rx_mean,index_group),\
                              value(Post_IDX_DS_mean,index_group),\
                              value(IDX_PDC_mean,index_group) ,\
                              value(IDX_Discon_mean,index_group) ])
    
    

for index_group in post_index_std.keys():
    post_index_std[index_group]=pd.Series([ \
                              value(Post_IDX_Rx_std,index_group),\
                              value(Post_IDX_DS_std,index_group),\
                              value(IDX_PDC_std,index_group) ,\
                              value(IDX_Discon_std,index_group) ])
    
post_index_table = pd.concat([post_index_mean['AD'], post_index_std['AD'], \
                              post_index_mean['AC'], post_index_std['AC'], \
                              post_index_mean['DC'] ,post_index_std['DC'], \
                              post_index_mean['Neither'], post_index_std['Neither']]\
                               , axis=1)

for column_num in range(8):
    post_index_table.iloc[:,column_num]=round(post_index_table.iloc[:,column_num] ,2)
   
    
post_index_table

,0,1,2,3,4,5,6,7
0,3.21,2.05,3.23,2.09,3.61,2.13,1.80,0.84
1,131.55,75.89,131.38,79.55,135.32,68.48,52.00,27.75
2,0.73,0.42,0.73,0.44,0.75,0.38,0.29,0.15
3,0.64,0.48,0.62,0.49,0.57,0.50,1.00,0.00


In [247]:
#representing numbers and percentages
result.index=pd.MultiIndex.from_arrays([['At Index']*7 + ['Pre-Index']*3 + \
                                        ['Post-Index']*2, \
                                        ['N' ,\
                                         'Male (n, %)',\
                                         'Age 0-18 at index (n,%)' ,\
                                         'Age 19-50 at index (n,%)' ,\
                                         'Age 51-65 at index (n,%)' ,\
                                         'Age 66-75 at index (n,%)' ,\
                                         'Age 76+ at index (n,%)' ,\
                                         'Pre_hosp (n,%)' ,\
                                         'Pre_Dyslipidemia (n,%)' ,\
                                         'Pre_HMG_CO-A (n,%)' ,\
                                         'IDX_PDC>80 (n, %)' ,\
                                         'Post_hosp_neg (n, %)' ]])

result.columns = pd.MultiIndex.from_product([['IDX_Group'],['Anti-Diabetic', 'Anticoagulant',\
                                                           'Both', 'Neither'], ["n", "%"]])

result

IDX_Group                        \
                                    Anti-Diabetic         Anticoagulant   
                                                n       %             n   
At Index   N                                 4958  100.00           806   
           Male (n, %)                       2401   48.43           434   
           Age 0-18 at index (n,%)            136    2.74             5   
           Age 19-50 at index (n,%)          1357   27.37           122   
           Age 51-65 at index (n,%)          2419   48.79           310   
           Age 66-75 at index (n,%)           682   13.76           148   
           Age 76+ at index (n,%)             362    7.30           221   
Pre-Index  Pre_hosp (n,%)                     632   12.75           269   
           Pre_Dyslipidemia (n,%)            2689   54.24           358   
           Pre_HMG_CO-A (n,%)                2784   56.15           375   
Post-Index IDX_PDC>80 (n, %)                 2376   47.92           400   
           Post_hosp_neg (n, %)              4680   94.39           699   

                                                                         
                                            Both         Neither         
                                          %    n       %       n      %  
At Index   N                         100.00   28  100.00       5  100.0  
           Male (n, %)                53.85   14   50.00       1   20.0  
           Age 0-18 at index (n,%)     0.62    0    0.00       0    0.0  
           Age 19-50 at index (n,%)   15.14    7   25.00       4   80.0  
           Age 51-65 at index (n,%)   38.46   13   46.43       0    0.0  
           Age 66-75 at index (n,%)   18.36    5   17.86       0    0.0  
           Age 76+ at index (n,%)     27.42    3   10.71       1   20.0  
Pre-Index  Pre_hosp (n,%)             33.37   10   35.71       2   40.0  
           Pre_Dyslipidemia (n,%)     44.42   17   60.71       2   40.0  
           Pre_HMG_CO-A (n,%)         46.53   15   53.57       1   20.0  
Post-Index IDX_PDC>80 (n, %)          49.63   15   53.57       0    0.0  
           Post_hosp_neg (n, %)       86.72   27   96.43       4   80.0

In [248]:
#representing mean and standard deviation
post_index_table.index=pd.MultiIndex.from_arrays([['Post-Index']*4, \
                                        ['Post_IDX_Rx (Mean, SD)' ,\
                                         'Post_IDX_DS (Mean, SD)' ,\
                                         'IDX_PDC (Mean, SD)' ,\
                                         'IDX_Discon (Mean, SD)']])

post_index_table.columns = pd.MultiIndex.from_product([['IDX_Group'],['Anti-Diabetic', 'Anticoagulant',\
                                                           'Both', 'Neither'], ["Mean", "SD"]])

post_index_table

IDX_Group                              \
                                  Anti-Diabetic        Anticoagulant          
                                           Mean     SD          Mean     SD   
Post-Index Post_IDX_Rx (Mean, SD)          3.21   2.05          3.23   2.09   
           Post_IDX_DS (Mean, SD)        131.55  75.89        131.38  79.55   
           IDX_PDC (Mean, SD)              0.73   0.42          0.73   0.44   
           IDX_Discon (Mean, SD)           0.64   0.48          0.62   0.49   

                                                                 
                                     Both        Neither         
                                     Mean     SD    Mean     SD  
Post-Index Post_IDX_Rx (Mean, SD)    3.61   2.13    1.80   0.84  
           Post_IDX_DS (Mean, SD)  135.32  68.48   52.00  27.75  
           IDX_PDC (Mean, SD)        0.75   0.38    0.29   0.15  
           IDX_Discon (Mean, SD)     0.57   0.50    1.00   0.00